In [5]:


import tkinter as tk
from tkinter import messagebox
import numpy as np
import time

In [6]:
class Sudoku:
    def __init__(self, tablero):
        self.tablero = tablero
        self.tamano = 9

    def es_valido(self, fila, col, num):
        if num in self.tablero[fila]:
            return False
        if num in self.tablero[:, col]:
            return False
        inicio_fila = fila - fila % 3
        inicio_col = col - col % 3
        if num in self.tablero[inicio_fila:inicio_fila + 3, inicio_col:inicio_col + 3]:
            return False
        return True

    def obtener_casilla_vacia(self):
        for fila in range(self.tamano):
            for col in range(self.tamano):
                if self.tablero[fila][col] == 0:
                    return fila, col
        return None

    def resolver_sudoku(self):
        casilla = self.obtener_casilla_vacia()
        if not casilla:
            return True
        fila, col = casilla
        for num in range(1, 10):
            if self.es_valido(fila, col, num):
                self.tablero[fila][col] = num
                if self.resolver_sudoku():
                    return True
                self.tablero[fila][col] = 0
        return False

In [7]:
class SudokuApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Sudoku Solver")
        self.tablero = self.generar_tablero_inicial()
        self.entradas = [[None for _ in range(9)] for _ in range(9)]
        self.crear_widgets()

    def crear_widgets(self):
        colores = [
            '#FFDDC1', '#FFABAB', '#FFC3A0',
            '#FF677D', '#D4A5A5', '#392F5A',
            '#B9FBC0', '#A0D3E8', '#E3B23C'
        ]
        
        for fila in range(9):
            for col in range(9):
                entrada = tk.Entry(self.master, width=2, font=('Arial', 24), justify='center', borderwidth=2, relief='solid')
                entrada.grid(row=fila, column=col, padx=5, pady=5)

                entrada.config(bg=colores[(fila // 3) * 3 + (col // 3)])
                
                entrada.bind("<KeyRelease>", self.limitar_input)
                
                if self.tablero[fila][col] != 0:
                    entrada.insert(0, str(self.tablero[fila][col]))
                    entrada.config(state='readonly')
                self.entradas[fila][col] = entrada

        resolver_btn = tk.Button(self.master, text="Resolver Sudoku", command=self.resolver)
        resolver_btn.grid(row=10, column=0, columnspan=9)

    def limitar_input(self, event):
        entry = event.widget
        valor = entry.get()
        if valor.isdigit() and valor != "0":
            if len(valor) > 1:
                entry.delete(0, tk.END)
                entry.insert(0, valor[0])  # Asegurar que solo queda el primer dígito
            self.actualizar_tablero()
        else:
            entry.delete(0, tk.END)  # Borrar si no es un número válido

    def actualizar_tablero(self):
        for fila in range(9):
            for col in range(9):
                valor = self.entradas[fila][col].get()
                if valor.isdigit() and valor != "":
                    self.tablero[fila][col] = int(valor)
                else:
                    self.tablero[fila][col] = 0

    def intentar_resolver_sudoku(self):
        sudoku = Sudoku(np.copy(self.tablero))
        if sudoku.resolver_sudoku():
            self.tablero = sudoku.tablero
            self.mostrar_tablero()
            return True
        else:
            return False

    def resolver(self):
        inicio = time.time()
        if self.intentar_resolver_sudoku():
            fin = time.time()
            tiempo_total = fin - inicio
            messagebox.showinfo("Éxito", f"El Sudoku se resolvió en {tiempo_total:.4f} segundos")
        else:
            messagebox.showinfo("Error", "El Sudoku no tiene solución")

    def mostrar_tablero(self):
        for fila in range(9):
            for col in range(9):
                self.entradas[fila][col].delete(0, tk.END)
                if self.tablero[fila][col] != 0:
                    self.entradas[fila][col].insert(0, str(self.tablero[fila][col]))

    def generar_tablero_inicial(self):
        tablero_inicial = [
            [0, 2, 0, 0, 1, 3, 0, 6, 0],
            [0, 0, 5, 6, 0, 0, 3, 4, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0],
            [1, 0, 2, 0, 7, 0, 0, 8, 5],
            [0, 9, 0, 0, 0, 2, 0, 0, 0],
            [7, 0, 0, 0, 3, 0, 0, 0, 0],
            [0, 0, 0, 3, 0, 5, 9, 0, 0],
            [0, 0, 0, 0, 2, 0, 0, 5, 1],
            [0, 0, 0, 8, 0, 0, 0, 7, 0]
        ]
        return np.array(tablero_inicial)

In [8]:
if __name__ == "__main__":
    try:
        root = tk.Tk()
        app = SudokuApp(root)
        root.mainloop()
    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al iniciar la aplicación: {str(e)}")